In [1]:
import os

def list_files_and_dirs(path):
    try:
        items = os.listdir(path)
        for item in items:
            print(item)
    except FileNotFoundError:
        print(f"指定されたパスが見つかりません: {path}")
    except NotADirectoryError:
        print(f"指定されたパスはディレクトリではありません: {path}")
    except PermissionError:
        print(f"指定されたパスへのアクセス権がありません: {path}")

# 使用例
# list_files_and_dirs('/path/to/directory')

In [2]:
list_files_and_dirs('../../../../kuzushiji-recognition/char_sep_datas')


.DS_Store
100241706


In [3]:
import json
import os
from PIL import Image
import glob

def investigate_image_annotation_mismatch():
    """画像とアノテーションデータの対応関係を調査"""
    
    # パスの設定
    input_images_path = '../../../../kuzushiji-recognition/synthetic_images/input_images/'
    json_path = '../../../../kuzushiji-recognition/synthetic_images/gt_json.json'
    
    print("=== 画像とアノテーションデータの対応調査 ===\n")
    
    # 1. 画像ファイル一覧を取得
    if os.path.exists(input_images_path):
        image_files = glob.glob(os.path.join(input_images_path, "*.jpg"))
        image_ids = [os.path.basename(f).replace('.jpg', '') for f in image_files]
        print(f"画像ファイル数: {len(image_files)}")
        print(f"最初の5つの画像ID: {image_ids[:5]}")
    else:
        print(f"画像ディレクトリが見つかりません: {input_images_path}")
        return
    
    # 2. JSONファイルを読み込み
    if os.path.exists(json_path):
        with open(json_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
        
        json_file_ids = list(json_data.get('files', {}).keys())
        print(f"\nJSONファイル内のID数: {len(json_file_ids)}")
        print(f"最初の5つのJSONファイルID: {json_file_ids[:5]}")
    else:
        print(f"JSONファイルが見つかりません: {json_path}")
        return
    
    # 3. ID形式の分析
    print("\n=== ID形式の分析 ===")
    print("画像ファイルIDの例:")
    for i, img_id in enumerate(image_ids[:5]):
        print(f"  {i+1}: '{img_id}'")
    
    print("\nJSONファイルIDの例:")
    for i, json_id in enumerate(json_file_ids[:5]):
        print(f"  {i+1}: '{json_id}'")
    
    # 4. ID対応関係の確認
    print("\n=== ID対応関係の確認 ===")
    
    # 画像IDに対応するJSONエントリがあるかチェック
    missing_in_json = []
    for img_id in image_ids[:10]:  # 最初の10個をチェック
        if img_id in json_data.get('files', {}):
            print(f"✓ 画像ID '{img_id}' → JSON内に存在")
        else:
            # 類似IDを探す
            similar_ids = [jid for jid in json_file_ids if img_id in jid or jid in img_id]
            if similar_ids:
                print(f"✗ 画像ID '{img_id}' → JSON内に直接対応なし")
                print(f"    類似ID: {similar_ids[:3]}")
            else:
                print(f"✗ 画像ID '{img_id}' → JSON内に対応なし")
                missing_in_json.append(img_id)
    
    # 5. JSONのIDでファイル名変換規則を推測
    print("\n=== ID変換規則の推測 ===")
    if json_file_ids:
        sample_json_id = json_file_ids[0]
        print(f"JSONファイルID例: '{sample_json_id}'")
        
        # '_sep_' が含まれているかチェック
        if '_sep_' in sample_json_id:
            parts = sample_json_id.split('_sep_')
            print(f"  '_sep_'で分割: {parts}")
            if len(parts) == 2:
                doc_id, file_part = parts
                print(f"  推測される変換: doc_id='{doc_id}', file_part='{file_part}'")
                
                # 対応する画像ファイルを推測
                expected_image_id = file_part  # または別の変換規則
                if expected_image_id in image_ids:
                    print(f"  ✓ 対応画像ファイル見つかる: '{expected_image_id}'")
                else:
                    print(f"  ✗ 対応画像ファイル見つからない: '{expected_image_id}'")
    
    return {
        'image_ids': image_ids,
        'json_file_ids': json_file_ids,
        'missing_in_json': missing_in_json
    }

# 調査実行
result = investigate_image_annotation_mismatch()

=== 画像とアノテーションデータの対応調査 ===

画像ファイル数: 76
最初の5つの画像ID: ['100241706_sep_100241706_00027_1', '100241706_sep_100241706_00018_2', '100241706_sep_100241706_00003_1', '100241706_sep_100241706_00025_2', '100241706_sep_100241706_00027_2']

JSONファイル内のID数: 76
最初の5つのJSONファイルID: ['100241706_sep_100241706_00003_1', '100241706_sep_100241706_00025_2', '100241706_sep_100241706_00018_2', '100241706_sep_100241706_00027_1', '100241706_sep_100241706_00025_1']

=== ID形式の分析 ===
画像ファイルIDの例:
  1: '100241706_sep_100241706_00027_1'
  2: '100241706_sep_100241706_00018_2'
  3: '100241706_sep_100241706_00003_1'
  4: '100241706_sep_100241706_00025_2'
  5: '100241706_sep_100241706_00027_2'

JSONファイルIDの例:
  1: '100241706_sep_100241706_00003_1'
  2: '100241706_sep_100241706_00025_2'
  3: '100241706_sep_100241706_00018_2'
  4: '100241706_sep_100241706_00027_1'
  5: '100241706_sep_100241706_00025_1'

=== ID対応関係の確認 ===
✓ 画像ID '100241706_sep_100241706_00027_1' → JSON内に存在
✓ 画像ID '100241706_sep_100241706_00018_2' → JSON内に存在
✓ 

In [4]:
def investigate_id_generation_process():
    """ID生成過程の問題を調査"""
    
    print("=== ID生成過程の調査 ===\n")
    
    # generate_dateset.pyのID生成ロジックを再現
    def simulate_id_generation(file_path):
        """ID生成ロジックのシミュレーション"""
        doc_id = file_path.split('/')[-3]
        file_id = str(doc_id) + '_sep_' + file_path.split('/')[-1].split('.')[0]
        return file_id
    
    # サンプルファイルパスでテスト
    sample_paths = [
        '/path/to/100241706/images/100241706_00003_1.jpg',
        '/path/to/100241706/images/100241706_00025_2.jpg',
        '/path/to/100249371/images/100249371_00001_1.jpg'
    ]
    
    print("ID生成シミュレーション:")
    for path in sample_paths:
        generated_id = simulate_id_generation(path)
        print(f"  パス: {path}")
        print(f"  生成ID: '{generated_id}'")
        print()
    
    return

def check_concurrent_processing_issues():
    """並行処理での問題を調査"""
    
    print("=== 並行処理問題の調査 ===\n")
    
    json_path = '../../../../kuzushiji-recognition/synthetic_images/gt_json.json'
    
    if not os.path.exists(json_path):
        print(f"JSONファイルが見つかりません: {json_path}")
        return
    
    with open(json_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
    
    # 1. 同じdoc_idを持つエントリを分析
    doc_id_groups = {}
    for file_id in json_data.get('files', {}).keys():
        if '_sep_' in file_id:
            doc_id = file_id.split('_sep_')[0]
            if doc_id not in doc_id_groups:
                doc_id_groups[doc_id] = []
            doc_id_groups[doc_id].append(file_id)
    
    print("doc_id別のファイル数:")
    for doc_id, file_ids in doc_id_groups.items():
        print(f"  {doc_id}: {len(file_ids)}個のファイル")
        if len(file_ids) <= 5:  # 少ない場合は全て表示
            for fid in file_ids:
                print(f"    - {fid}")
        else:  # 多い場合は最初の3個だけ
            for fid in file_ids[:3]:
                print(f"    - {fid}")
            print(f"    ... 他{len(file_ids)-3}個")
        print()
    
    # 2. アノテーションデータの内容をチェック
    print("=== アノテーションデータの内容チェック ===")
    sample_ids = list(json_data.get('files', {}).keys())[:3]
    
    for file_id in sample_ids:
        data = json_data['files'][file_id]
        print(f"ファイルID: {file_id}")
        print(f"  main_region数: {len(data.get('main_region', []))}")
        print(f"  main_affinity数: {len(data.get('main_affinity', []))}")
        print(f"  furi_region数: {len(data.get('furi_region', []))}")
        print(f"  furi_affinity数: {len(data.get('furi_affinity', []))}")
        
        # 座標データの妥当性チェック
        if data.get('main_region'):
            first_region = data['main_region'][0]
            print(f"  最初のregion座標: {first_region}")
            
            # 座標が画像サイズ内に収まっているかチェック
            # (通常は0-数千の範囲)
            coords = [float(x) for x in first_region]
            if any(c < 0 or c > 10000 for c in coords):
                print(f"  ⚠️ 異常な座標値が検出されました: {coords}")
        print()
    
    return doc_id_groups

# 調査実行
investigate_id_generation_process()
doc_groups = check_concurrent_processing_issues()

=== ID生成過程の調査 ===

ID生成シミュレーション:
  パス: /path/to/100241706/images/100241706_00003_1.jpg
  生成ID: '100241706_sep_100241706_00003_1'

  パス: /path/to/100241706/images/100241706_00025_2.jpg
  生成ID: '100241706_sep_100241706_00025_2'

  パス: /path/to/100249371/images/100249371_00001_1.jpg
  生成ID: '100249371_sep_100249371_00001_1'

=== 並行処理問題の調査 ===

doc_id別のファイル数:
  100241706: 76個のファイル
    - 100241706_sep_100241706_00003_1
    - 100241706_sep_100241706_00025_2
    - 100241706_sep_100241706_00018_2
    ... 他73個

=== アノテーションデータの内容チェック ===
ファイルID: 100241706_sep_100241706_00003_1
  main_region数: 361
  main_affinity数: 342
  furi_region数: 786
  furi_affinity数: 472
  最初のregion座標: [222, 430, 347, 430, 347, 549, 222, 549]

ファイルID: 100241706_sep_100241706_00025_2
  main_region数: 155
  main_affinity数: 142
  furi_region数: 278
  furi_affinity数: 139
  最初のregion座標: [393, 521, 509, 521, 509, 621, 393, 621]

ファイルID: 100241706_sep_100241706_00018_2
  main_region数: 251
  main_affinity数: 234
  furi_region数: 453
  

In [6]:
# generate_dateset.pyの並行処理問題を詳細調査
def analyze_concurrent_processing_risks():
    """generate_dateset.pyの並行処理リスクを分析"""
    
    print("=== 並行処理リスク分析 ===\n")
    
    print("🔍 generate_dateset.pyの並行処理構造:")
    print("""
    main関数:
    ├── ProcessPoolExecutor(max_workers=20)  ← 20並列プロセス
    │   ├── process1: main_exe_for_one_image()
    │   ├── process2: main_exe_for_one_image()
    │   ├── ...
    │   └── process20: main_exe_for_one_image()
    │
    └── 各プロセスが同じJSONファイルに書き込み
        └── update_json_data() ← ここで競合発生の可能性
    """)
    
    print("⚠️ 特定された問題点:\n")
    
    print("1. 【ファイルロック競合】")
    print("   - 20個のプロセスが同時に同じJSONファイルにアクセス")
    print("   - 'with lock:' はプロセス間ロックではなくスレッド間ロック")
    print("   - multiprocessing.Manager().Lock() が必要\n")
    
    print("2. 【JSONファイル全体読み書き】")
    print("   - update_json_data()で毎回JSONファイル全体を読み込み")
    print("   - json_data['files'][file_id] = data で上書き")
    print("   - 同時書き込みでデータが失われる可能性\n")
    
    print("3. 【レースコンディション】")
    print("   プロセスA: JSONファイル読み込み → 処理中")
    print("   プロセスB: 同じJSONファイル読み込み → 処理中")
    print("   プロセスA: データ追加して保存")
    print("   プロセスB: 古いデータに追加して保存 ← プロセスAの変更が消失\n")
    
    print("4. 【ID重複の可能性】")
    print("   - ファイルパス解析でのdoc_id取得")
    print("   - 同じdoc_idの異なるファイルの処理タイミング")
    print("   - 並行処理での上書き競合\n")

def simulate_race_condition():
    """レースコンディションのシミュレーション"""
    
    print("=== レースコンディション シミュレーション ===\n")
    
    print("【正常な処理順序】")
    print("時刻1: プロセスA - JSONファイル読み込み {}")
    print("時刻2: プロセスA - image1のデータ処理")
    print("時刻3: プロセスA - JSONに{'image1': data1}追加保存")
    print("時刻4: プロセスB - JSONファイル読み込み {'image1': data1}")
    print("時刻5: プロセスB - image2のデータ処理")
    print("時刻6: プロセスB - JSONに{'image1': data1, 'image2': data2}保存\n")
    
    print("【問題のある処理順序（レースコンディション）】")
    print("時刻1: プロセスA - JSONファイル読み込み {}")
    print("時刻2: プロセスB - JSONファイル読み込み {} (同じ空データ)")
    print("時刻3: プロセスA - image1のデータ処理")
    print("時刻4: プロセスB - image2のデータ処理")
    print("時刻5: プロセスA - JSONに{'image1': data1}保存")
    print("時刻6: プロセスB - JSONに{'image2': data2}保存 ← image1のデータが消失!")
    print("時刻7: または、プロセスBがimage1用のデータをimage2に誤って結合\n")
    
    print("【結果】")
    print("- image1の画像に、image2のアノテーションが結び付く")
    print("- または、image1のアノテーションが完全に失われる")
    print("- JSONファイルにはimage2のデータのみ残る\n")

def recommend_solutions():
    """解決策の提案"""
    
    print("=== 解決策の提案 ===\n")
    
    print("🔧 【緊急対策】")
    print("1. 並行処理を無効化")
    print("   - max_workers=1 に設定")
    print("   - または ProcessPoolExecutor を使わず順次処理\n")
    
    print("2. プロセス間ロックの実装")
    print("""
    from multiprocessing import Manager
    
    if __name__ == "__main__":
        with Manager() as manager:
            file_lock = manager.Lock()  # プロセス間ロック
            with ProcessPoolExecutor(max_workers=20) as executor:
                futures = []
                for file_path in file_path_list:
                    futures.append(executor.submit(
                        main_exe_for_one_image,
                        file_path=file_path,
                        json_gt=json_gt,
                        file_lock=file_lock  # ロックを渡す
                    ))
    """)
    
    print("3. 個別JSONファイル方式")
    print("   - 各画像のアノテーションを個別ファイルに保存")
    print("   - 最後に全てを統合")
    print("   - 例: image1.json, image2.json → 最終的に統合\n")
    
    print("🔧 【根本的解決策】")
    print("1. データベース使用 (SQLite)")
    print("2. キュー方式での順次書き込み")
    print("3. 事前にファイルIDリストを作成し、重複チェック\n")

# 分析実行
analyze_concurrent_processing_risks()
simulate_race_condition()
recommend_solutions()

=== 並行処理リスク分析 ===

🔍 generate_dateset.pyの並行処理構造:

    main関数:
    ├── ProcessPoolExecutor(max_workers=20)  ← 20並列プロセス
    │   ├── process1: main_exe_for_one_image()
    │   ├── process2: main_exe_for_one_image()
    │   ├── ...
    │   └── process20: main_exe_for_one_image()
    │
    └── 各プロセスが同じJSONファイルに書き込み
        └── update_json_data() ← ここで競合発生の可能性
    
⚠️ 特定された問題点:

1. 【ファイルロック競合】
   - 20個のプロセスが同時に同じJSONファイルにアクセス
   - 'with lock:' はプロセス間ロックではなくスレッド間ロック
   - multiprocessing.Manager().Lock() が必要

2. 【JSONファイル全体読み書き】
   - update_json_data()で毎回JSONファイル全体を読み込み
   - json_data['files'][file_id] = data で上書き
   - 同時書き込みでデータが失われる可能性

3. 【レースコンディション】
   プロセスA: JSONファイル読み込み → 処理中
   プロセスB: 同じJSONファイル読み込み → 処理中
   プロセスA: データ追加して保存
   プロセスB: 古いデータに追加して保存 ← プロセスAの変更が消失

4. 【ID重複の可能性】
   - ファイルパス解析でのdoc_id取得
   - 同じdoc_idの異なるファイルの処理タイミング
   - 並行処理での上書き競合

=== レースコンディション シミュレーション ===

【正常な処理順序】
時刻1: プロセスA - JSONファイル読み込み {}
時刻2: プロセスA - image1のデータ処理
時刻3: プロセスA - JSONに{'image1': data1}追加保存
時刻4: プロ

In [7]:
def emergency_fix_suggestion():
    """緊急対策の具体的な修正提案"""
    
    print("=== 緊急対策: generate_dateset.py の修正 ===\n")
    
    print("🚨 現在のコード (問題あり):")
    print("""
    if __name__ == "__main__":
        with Manager() as manager:
            with ProcessPoolExecutor(max_workers=20) as executor:  # ← 20並列
                for doc_path in doc_path_list:
                    for file_path in file_path_list:
                        if procedure_for_one_image != None:
                            futures.append(executor.submit(
                                main_exe_for_one_image, 
                                file_path=file_path,
                                json_gt=json_gt,
                                procedure_for_one_image=procedure_for_one_image
                            ))
    """)
    
    print("✅ 修正版1: 並行処理無効化")
    print("""
    if __name__ == "__main__":
        with Manager() as manager:
            with ProcessPoolExecutor(max_workers=1) as executor:  # ← 1に変更
                for doc_path in doc_path_list:
                    for file_path in file_path_list:
                        if procedure_for_one_image != None:
                            futures.append(executor.submit(
                                main_exe_for_one_image, 
                                file_path=file_path,
                                json_gt=json_gt,
                                procedure_for_one_image=procedure_for_one_image
                            ))
    """)
    
    print("✅ 修正版2: 完全に順次処理")
    print("""
    if __name__ == "__main__":
        for doc_path in doc_path_list:
            for file_path in file_path_list:
                if procedure_for_one_image != None:
                    # 直接実行（並行処理なし）
                    main_exe_for_one_image(
                        file_path=file_path,
                        json_gt=json_gt,
                        procedure_for_one_image=procedure_for_one_image
                    )
    """)
    
    print("⚠️ 注意点:")
    print("- 処理時間は20倍長くなりますが、データの整合性が保たれます")
    print("- 既存のJSONファイルはバックアップを取ってから再生成してください")
    print("- 長期的にはプロセス間ロックまたは個別ファイル方式への移行を推奨\n")
    
    print("🔧 バックアップ手順:")
    print("1. 現在のgt_json.jsonをバックアップ")
    print("2. generate_dateset.pyを修正版で実行") 
    print("3. 結果を確認して問題がないことを検証")
    
emergency_fix_suggestion()

=== 緊急対策: generate_dateset.py の修正 ===

🚨 現在のコード (問題あり):

    if __name__ == "__main__":
        with Manager() as manager:
            with ProcessPoolExecutor(max_workers=20) as executor:  # ← 20並列
                for doc_path in doc_path_list:
                    for file_path in file_path_list:
                        if procedure_for_one_image != None:
                            futures.append(executor.submit(
                                main_exe_for_one_image, 
                                file_path=file_path,
                                json_gt=json_gt,
                                procedure_for_one_image=procedure_for_one_image
                            ))
    
✅ 修正版1: 並行処理無効化

    if __name__ == "__main__":
        with Manager() as manager:
            with ProcessPoolExecutor(max_workers=1) as executor:  # ← 1に変更
                for doc_path in doc_path_list:
                    for file_path in file_path_list:
                        if procedure_for_one_ima